<a href="https://colab.research.google.com/github/tarabelo/PIAC-2526/blob/main/Computaci%C3%B3n%20cu%C3%A1ntica%20adiab%C3%A1tica%20y%20Quantum%20Annealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalamos qiskit en el notebook
!pip install qiskit qiskit-aer qiskit-algorithms qiskit-optimization qiskit_machine_learning qiskit-ibm-runtime pylatexenc

In [ ]:
import numpy as np
from math import sqrt

# importing Qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator, StatevectorSimulator

# import basic plot tools
from qiskit.visualization import plot_histogram

# Funciones auxiliares

# Función para obtener y mostrar el vector de estado
def obten_estado(qcirc, etiqueta="|\psi\!\!> = ", bloch=False):
  estado = Statevector.from_instruction(qcirc)
  display(estado.draw('latex', prefix=etiqueta, max_size=2**qc.num_qubits))
  if bloch:
    display(estado.draw('bloch'))

# **Computación cuántica adiabática y Quantum Annealing**

### Contenidos

1. [Computación cuántica adiabática](#adiabatica)
1. [Quantum Annealing](#annealing)
1. [Variational Quantum Eigensolvers (VQE)](#vqe)


<a name="adiabatica"></a>

## **Computación cuántica adiabática**

Forma alternativa de computación cuántica basada en la evolución temporal de un Hamiltoniano y en el [teorema adiabático](https://en.wikipedia.org/wiki/Adiabatic_theorem)

#### Ecuación de Schrödinger

La evolución temporal de un sistema cuántico viene dada por la ecuación de Schrödinger:

$$i\hbar \frac{\partial}{\partial t}|\psi(t)\rangle = H(t)|\psi(t)\rangle$$

$H(t)$ es el Hamiltoniano del sistema en el instante $t$ y define la energía del mismo.

#### Teorema adiabático
Si un sistema cuántico está en un estado base (estado de menor energía o _ground state_) de un Hamiltoniano inicial y se le hace evolucionar muy lentamente hacia un Hamiltoniano final, este estado inicial evolucionará al estado base del Hamiltoniano final, tras un tiempo suficientemente largo.

Supongamos que tenemos un Hamiltoniano $H_c$ (_Hamiltoniano de coste_) cuyo estado base codifica la solución de nuestro problema.

Partimos de un Hamiltoniano $H_0$ cuyo estado base es facil de preparar y queremos llegar, en un tiempo $T$, al estado base de $H_c$. Hacemos evolucionar el sistema desde $t=0$ hasta $t=T$ como sigue:

$$H(t) = A(t)H_0 + B(t)H_c$$

donde las funciones $A$ y $B$ verifican: $A(0) = B(T) = 1$ y $A(T) = B(0) = 0$.

Un ejemplo de estas funciones son

$$
A(t) =  \left(1-\frac{t}{T}\right)\\
B(t) = \frac{t}{T}
$$


Si $t$ cambia muy lentamente, el sistema pasa de $H(0) = H_0$ a $H(T) = H_c$ manteniéndose en el estado base.

Midiendo el estado del sistema en $t=T$ obtenemos el estado $|\psi\rangle$ que minimiza $\langle\psi|H_c|\psi\rangle$ y que es la solución del problema.

En [Aharonov et al. 2007](https://doi.org/10.1137/080734479) se demostró que la computación cuántica adiabática es equivalente a la computación cuántica basada en puertas.

#### Tiempo de evolución

El teorema adiabático establece que para que $H(t)$ se mantenga en el estado base, $T$ tiene que ser inversamente proporcional al cuadrado del *spectral gap* (minima diferencia de energía entre el estado base y el primer estado excitado de $H(t)$ con $t\in[0,T]$).

Encontrar el valor del *spectral gap* es muy complejo. Además, $T$ podría llegar a ser muy grande. El *Quantum annealing* es una implementación práctica de la computación cuántica adiabática.




<a name="annealing"></a>

## **Quantum annealing**

Es un modelo similar a la computación cuántica adiabática, con ciertas limitaciones:

- El Hamiltoniano final tiene que ser un Hamiltoniano Ising:  

$$
H_c = -\sum_{i,j=0}^{n-1} J_{ij}Z_iZ_j - \sum_{i=0}^{n-1} h_{i}Z_i
$$

- No se mantiene la evolución adiabática $\implies$ el estado final podría no ser el estado base de $H_c$

**Estado inicial $H_0$**

Se suele usar como estado inicial el siguiente:

$$
H_0 = -\sum_{i=0}^{n-1} X_i
$$

Este estado se denomina _mixing Hamiltonian_ y su estado base es $|\psi_0\rangle = |+\rangle^{\otimes n}$

**Evolución del Hamiltoniano**

La evolución viene dada por:

$$
H(t) = A(t)H_0 + B(t)H_c = -A(t)\sum_{i=0}^{n-1} X_i - B(t)\sum_{i,j=0}^{n-1} J_{ij}Z_iZ_j - B(t)\sum_{i=0}^{n-1} h_{i}Z_i
$$


### Quantum annealers
Computadores cuánticos que resuelven problemas de optimización mediante Quantum annealing.

- [D-Wave Systems](https://www.dwavesys.com/): Primera empresa en vender quantum annealers
  - El último modelo dispone de más de 7000 cúbits
  - Suite [Ocean](https://docs.ocean.dwavesys.com/en/latest/)
  - Tipo de problemas: optimización de portfolios, optimización de rutas y gestión de flotas, optimización de líneas de producción, búsqueda de similaridad entre moléculas, etc.




In [ ]:
!pip install dwave-neal dimod

In [ ]:
# dimod: biblioteca base de D-Wave Ocean para representar
# y resolver problemas combinatorios binarios cuadráticos
from dimod import BinaryQuadraticModel, SPIN
J = {(0,1):1, (0,2):1}
h = {}

# Expresamos el grafo simple como un modelo binario cuadrático
# SPIN: las variables toman valores {-1,1}
problem = BinaryQuadraticModel(h, J, 0.0, SPIN)
print("The problem we are going to solve is:")
print(problem)

In [ ]:
# neal: simulador de annealing de DWave
from neal import SimulatedAnnealingSampler

# Usamos un simulador clásico en vez del D-WaveSampler
sampler = SimulatedAnnealingSampler()

# Ejecutamos la muestra con 10 repeticiones
result = sampler.sample(problem, num_reads=10)

print("\nThe solutions that we have obtained are:")
print(result)


In [ ]:
# Uso en un sistemna Dwave
# 1. Instalar el SDK de Ocean con pip install dwave-ocean-sdk
# 2. Registrarse en Leap (https://cloud.dwavesys.com/leap/) y obtener una clave API
# 3. Usar "dwave config create" para añadir la clave a nuestra configuración
#
# Código adaptado
#from dwave.system import DWaveSampler
#from dwave.system import EmbeddingComposite
#sampler = EmbeddingComposite(DWaveSampler())
#result = sampler.sample(problem, num_reads=10)
#print("The solutions that we have obtained are")
#print(result)



---



---



---

